# Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import random

In [3]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
# from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.linear_model import LogisticRegression
# from sklearn.neural_network import MLPClassifier, MLPRegressor
# from sklearn.svm import SVC
# from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold

In [4]:
from imblearn.under_sampling import RandomUnderSampler

In [5]:
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.pipeline import Pipeline

In [6]:
from collections import Counter

In [7]:
import wandb

In [8]:
from sklearn import preprocessing

In [9]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnxruntime as rt

In [10]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EXS

In [11]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

import torch.nn as nn
import torch.nn.functional as F

In [12]:
import torch.optim as optim

In [13]:
torch.__version__

'1.2.0'

In [14]:
from tqdm.notebook import tqdm

# Function

In [15]:
# Function(Utility)

# check Duplicate file
def checkDuplicateFile(file_path):
    import os
    if os.path.isfile(file_path):
        print("Caution: File existed!")
        ans = input("Do you want to cover it?(Y/others)")
        if ans == "Y":
            return False
        else:
            print("Canceled....")
            return True
    else:
        return False


# Function(Use)

# Test for checking ICU_id missing in Lab_1103_csv
def getMissingIDinLab(Lab_file, show=True):
    test = sorted(Lab_file.ICU_id.unique())
    s = 0
    error_list = list()
    for i in test:
        #         print(i)
        s += 1
        if s != i:
            if show:
                print(f"error! : {s}")
            error_list.append(s)
            s += 1
    if show:
        print(f"Missing ID Result: {error_list}")
    if show:
        print(f"Missing Length:{len(error_list)}")
    return error_list

# store Dataframe to CSV


def store2CSV(data, target_name, target_loc_prefix='./'):
    file_path = target_loc_prefix+target_name+".csv"
    if checkDuplicateFile(file_path):
        print("store2CSV failed")
        return
    data.to_csv(file_path)
    print("store2CSV Successful!")

# store Datastruc. to pickle


def store2Pickle(data, target_name, target_loc_prefix='./'):
    import pickle
    file_path = target_loc_prefix+target_name+'.pickle'
    if checkDuplicateFile(file_path):
        print("store2Pickle failed")
        return
    with open(file_path, 'wb') as f:
        pickle.dump(data, f)
    print("store2Pickle Successful!")


def readFPickle(target_name, target_loc_prefix='./'):
    import pickle
    file_name = target_loc_prefix+target_name+'.pickle'
    with open(file_name, 'rb') as f:
        temp = pickle.load(f)
    return temp

# Function(Data preprocessing)

#Function: 補值
# 將針對輸入的df_data直接進行inplace插補
# 須確保df_data的缺失值位置有放np.nan


def handleMissing(df_data, df_feature, outFeature=["outcome"], cate_astype="int"):
    for featureName in df_data.columns:
        if featureName not in outFeature:
            if df_data[featureName].isna().sum() == 0:
                print(f"{featureName}: Not need to fill.")
                continue
            else:
                # 先去看是連續與否 (1代表連續,0代表離散)
                kindValue = df_feature.loc[df_feature["features name"]
                                           == featureName, "kind"].values[0]
                if kindValue == 1:
                    # continuous
                    # mean filling
                    targetMean = df_data[featureName].mean()
                    df_data[featureName].fillna(value=targetMean, inplace=True)
                    print(f"{featureName}: Fill, Continuous.")

                else:
                    # categorical
                    # mode filling
                    targetMode = df_data[featureName].mode()[0]
                    df_data[featureName].fillna(value=targetMode, inplace=True)
                    df_data[featureName] = df_data[featureName].astype(
                        cate_astype)
                    print(
                        f"{featureName}: Fill, Categorical. (astype to {cate_astype})")

    print("---handleMissing Finish---")

# plot hist
# filtered_data need to check not have nan


def plotHist(df_data, target, outcome="outcome", bins=20):
    filtered_data = pd.concat([df_data["outcome"], df_data[target]], axis=1)
    filtered_data = filtered_data.dropna()
    print(filtered_data.isna().sum())
    plt.hist(filtered_data.loc[filtered_data[outcome]
             == 0, target], bins=bins, alpha=0.5, label='0')
    plt.hist(filtered_data.loc[filtered_data[outcome]
             == 1, target], bins=bins, alpha=0.5, label='1')
    plt.xlabel(target)
    plt.ylabel('count')
    plt.legend(title=outcome)

# plot countplot
# filtered_data need to check not have nan


def plotCountplot(df_data, target, outcome="outcome"):
    filtered_data = pd.concat([df_data["outcome"], df_data[target]], axis=1)
    filtered_data = filtered_data.dropna()
    print(filtered_data.isna().sum())
    sns.countplot(x=target, hue=outcome, data=filtered_data)

# plot boxplot
# filtered_data need to check not have nan


def plotBoxplot(df_data, target, outcome="outcome"):
    filtered_data = pd.concat([df_data["outcome"], df_data[target]], axis=1)
    filtered_data = filtered_data.dropna()
    print(filtered_data.isna().sum())
    sns.boxplot(x=filtered_data[target], data=filtered_data)

# Function(Model)


def serializeModel(model, modelName, featureNum):
    initial_type = [('float_input', FloatTensorType([None, featureNum]))]
    onx = convert_sklearn(model, initial_types=initial_type)
    with open(modelName + ".onnx", "wb") as f:
        f.write(onx.SerializeToString())


def modelPredict(modelName, testData):
    sess = rt.InferenceSession(modelName + '.onnx')  # load the onnx
    input_name = sess.get_inputs()[0].name
    label_name = sess.get_outputs()[0].name
    pred_onx = sess.run([label_name], {input_name: testData.astype(np.float32)})[
        0]  # predict testData
    print(pred_onx)

In [16]:
def get_data(X, y):
    X = torch.from_numpy(X).float()#transform the data from numpy to torch
    y = torch.from_numpy(y).long()
    
    dataset = TensorDataset(X,y)
    
    return dataset


def make_loader(dataset, batch_size):
    loader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=batch_size, 
                                         shuffle=True,
                                         pin_memory=True, num_workers=2)
    return loader

In [17]:
# ConvNet(self, kernel_size->list, input_size, classes=2)
def make(config, X_train, y_train, X_val, y_val):
    # Make the data
    # X_train, y_train, X_val, y_val is dataframe
    # now to ndarray
    X_train = X_train.values
    y_train = y_train.values
    X_val = X_val.values
    y_val = y_val.values
    train, test = get_data(X_train, y_train), get_data(X_val, y_val)
    train_loader = make_loader(train, batch_size=config.batch_size)
    test_loader = make_loader(test, batch_size=config.batch_size)

    # Make the model
    model = ConvNet(config.kernel_size, config.feature_size ,config.classes).to(device)

    # Make the loss and optimizer
    criterion = nn.CrossEntropyLoss()
#     optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
    optimizer = optim.SGD(model.parameters(), lr=config.learning_rate, momentum=0.9)

    return model, train_loader, test_loader, criterion, optimizer

In [18]:
def model_pipeline(hyperparameters, X_train, y_train, X_val, y_val):

    # tell wandb to get started
    with wandb.init(project="DataMining_Project2", entity="oscarchencs10", config=hyperparameters):
        # access all HPs through wandb.config, so logging matches execution!
        config = wandb.config

        # make the model, data, and optimization problem
        model, train_loader, test_loader, criterion, optimizer = make(config, X_train, y_train, X_val, y_val)
        print(model)

        # and use them to train the model
        train(model, train_loader, test_loader,criterion, optimizer, config)

        # and test its final performance
        test(model, test_loader)

    return model

# Train_log

In [19]:
# {"acc": correct / total, "f1":f1Score, "auc":auc}
def train_log(loss, example_ct, epoch, metrics, now_batch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss, "training_acc":metrics["acc"], 
               "training_f1":metrics["f1"], "training_auc":metrics["auc"]}, step=example_ct)
    print(f"----Epoch[{epoch}],Batch[{now_batch}]----")
    acc = metrics["acc"]
    f1 = metrics["f1"]
    auc = metrics["auc"]
    print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")
    print(f"Acc after " + str(example_ct).zfill(5) + f" examples: {acc:.4f}")
    print(f"f1_score after " + str(example_ct).zfill(5) + f" examples: {f1:.4f}")
    print(f"AUC after " + str(example_ct).zfill(5) + f" examples: {auc:.4f}")
    print(f"<-------------------------------------->")

# Model Architecture

In [20]:
# Conventional and convolutional neural network

class ConvNet(nn.Module):
    def __init__(self, kernel_size, input_size, classes=2):
        super(ConvNet, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=6, kernel_size=kernel_size[0]),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=1))
        self.layer2 = nn.Sequential(
            nn.Conv1d(in_channels=6, out_channels=16, kernel_size=kernel_size[1]),
            nn.ReLU(),)
        
        self.flatten = nn.Flatten()
        
        self.fc = nn.Sequential(
            nn.Linear(16*(input_size-kernel_size[-1]+1), 120),
            nn.ReLU(),
            nn.Linear(120, 32),
            nn.ReLU(),
            nn.Linear(32, classes),
        )
        
    def forward(self, x):
#         print(x.shape)
        x = torch.reshape(x, (-1,1, 40))
#         print(x.shape)
        out = self.layer1(x)
#         print(out.shape)
        out = self.layer2(out)
        out = self.flatten(out)
#         print(out.shape)
        out = self.fc(out)
#         print(out.shape)
        return out

# Train/Test

In [21]:
# {"acc": correct / total, "f1":f1_score, "auc":auc}
def train(model, loader, test_loader, criterion, optimizer, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    total_batches = len(loader) * config.epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0
    for epoch in tqdm(range(config.epochs)):
        now_batch = 0
        for _, (data_row, labels) in enumerate(loader):

            loss = train_batch(data_row, labels, model, optimizer, criterion)
            example_ct +=  len(data_row)
            batch_ct += 1
            now_batch += 1

            # Report metrics every 25th batch
            if ((batch_ct + 1) % 25) == 0:
                metrics = test(model, test_loader, training_phase=True)
                train_log(loss, example_ct, epoch, metrics, now_batch)


def train_batch(data_row, labels, model, optimizer, criterion):
    data_row, labels = data_row.to(device), labels.to(device)
    model.train()
    # Forward pass ➡
    outputs = model(data_row)
    loss = criterion(outputs, labels)
    
    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss

In [22]:
def test(model, test_loader, training_phase=False):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        prob_all = []
        prob_all2 = []
        label_all = []
        for data, labels in test_loader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1) #predicted:預測標籤結果
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            prob_all2.extend(outputs[:,1].cpu().numpy())
            prob_all.extend(predicted.cpu().numpy())
            label_all.extend(labels.cpu().numpy())

        f1Score = f1_score(label_all, prob_all)
        auc = roc_auc_score(label_all,prob_all2)
        print(f"Accuracy of the model on the {total} " +
              f"test data: {100 * correct / total}%")
        print(f"F1-Score of the model on the {total} " +
              "test data:{:.4f}".format(f1Score))
        print(f"AUC of the model on the {total} " +
              "test data:{:.4f}".format(auc))
        
        if not training_phase:
            wandb.log({"test_accuracy": correct / total, "test_f1":f1Score, "test_auc":auc})
            return None
        else:
            return {"acc": correct / total, "f1":f1Score, "auc":auc}

#     # Save the model in the exchangeable ONNX format
#     torch.onnx.export(model, images, "model.onnx")
#     wandb.save("model.onnx")

# Load Data

In [23]:
exp13_X_train = readFPickle("data/exp13/exp13_X_train")
exp13_X_train

,AGE,SEX,LOS,Joint,Drain,Cemented,Commercial_ALBC,Non_commercial_ALBC,cci_index,elx_index,...,Coagulopathy,Fluid and Electrolyte Disorders,Blood Loss Anemia,Deficiency Anemia,Anemia,Alcohol Abuse,Drug Abuse,Psychoses,Depression,Psyciatric disorder
25753,66,0,6,0,1,1,1,0,4,3,...,0,0,0,0,0,0,0,0,1,1
628,61,0,10,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
26083,80,1,2,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
37780,67,0,6,0,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
28632,62,0,4,1,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,70,0,7,0,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
44732,72,0,3,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
38158,68,0,6,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
860,31,1,4,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
exp13_y_train = readFPickle("data/exp13/exp13_y_train")
exp13_y_train

25753    0
628      0
26083    0
37780    1
28632    0
        ..
11284    0
44732    1
38158    1
860      0
15795    0
Name: outcome, Length: 45627, dtype: int64

In [25]:
exp13_X_val = readFPickle("data/exp13/exp13_X_val")
exp13_X_val

,AGE,SEX,LOS,Joint,Drain,Cemented,Commercial_ALBC,Non_commercial_ALBC,cci_index,elx_index,...,Coagulopathy,Fluid and Electrolyte Disorders,Blood Loss Anemia,Deficiency Anemia,Anemia,Alcohol Abuse,Drug Abuse,Psychoses,Depression,Psyciatric disorder
286,72.0,0,13,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
40555,78.0,1,3,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
28125,69.0,0,8,1,1,0,0,0,2,2,...,0,0,0,0,0,0,0,0,0,0
44616,72.0,1,7,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49606,80.0,0,3,1,0,0,0,0,2,3,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23499,81.0,1,5,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
47449,71.0,0,7,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
10480,55.0,0,3,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
35724,73.0,0,7,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
exp13_y_val = readFPickle("data/exp13/exp13_y_val")
exp13_y_val

286      1
40555    0
28125    0
44616    0
49606    0
        ..
23499    0
47449    0
10480    0
35724    0
5180     0
Name: outcome, Length: 8037, dtype: int64

# Random Seed

In [27]:
# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

# GPU Setting

In [28]:
# Device configuration
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=1)

In [29]:
torch.cuda.current_device()

0

In [30]:
torch.cuda.device_count()

4

In [31]:
torch.cuda.get_device_name(1)

'Tesla K40m'

In [32]:
torch.cuda.set_device(1)

In [33]:
torch.cuda.current_device()

1

# Config

In [34]:
config = dict(
    epochs=100,
    classes=2,
    kernel_size=[1, 1],
    feature_size=40,
    batch_size=128,
    learning_rate=0.005,
    dataset="PJI",
    architecture="CNN")

# Main

In [ ]:
# Build, train and analyze the model with the pipeline
model = model_pipeline(config, exp13_X_train, exp13_y_train, exp13_X_val, exp13_y_val)

wandb: Currently logged in as: oscarchencs10 (use `wandb login --relogin` to force relogin)


ConvNet(
  (layer1): Sequential(
    (0): Conv1d(1, 6, kernel_size=(1,), stride=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv1d(6, 16, kernel_size=(1,), stride=(1,))
    (1): ReLU()
  )
  (flatten): Flatten()
  (fc): Sequential(
    (0): Linear(in_features=640, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=2, bias=True)
  )
)


  0%|          | 0/100 [00:00<?, ?it/s]

Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.6822
----Epoch[0],Batch[24]----
Loss after 03072 examples: 0.620
Acc after 03072 examples: 0.9788
f1_score after 03072 examples: 0.0000
AUC after 03072 examples: 0.6822
<-------------------------------------->
Accuracy of the model on the 8037 test data: 93.30595993529924%
F1-Score of the model on the 8037 test data:0.0756
AUC of the model on the 8037 test data:0.6719
----Epoch[0],Batch[49]----
Loss after 06272 examples: 0.622
Acc after 06272 examples: 0.9331
f1_score after 06272 examples: 0.0756
AUC after 06272 examples: 0.6719
<-------------------------------------->
Accuracy of the model on the 8037 test data: 97.15067811372403%
F1-Score of the model on the 8037 test data:0.0255
AUC of the model on the 8037 test data:0.6684
----Epoch[0],Batch[74]----
Loss after 09472 examples: 0.603
Acc after 09472 examples: 0.9715
f1_score after

Accuracy of the model on the 8037 test data: 97.73547343536146%
F1-Score of the model on the 8037 test data:0.0215
AUC of the model on the 8037 test data:0.6810
----Epoch[1],Batch[599]----
Loss after 76603 examples: 0.574
Acc after 76603 examples: 0.9774
f1_score after 76603 examples: 0.0215
AUC after 76603 examples: 0.6810
<-------------------------------------->
Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.6698
----Epoch[1],Batch[624]----
Loss after 79803 examples: 0.587
Acc after 79803 examples: 0.9788
f1_score after 79803 examples: 0.0000
AUC after 79803 examples: 0.6698
<-------------------------------------->
Accuracy of the model on the 8037 test data: 97.88478287918377%
F1-Score of the model on the 8037 test data:0.0000
AUC of the model on the 8037 test data:0.6712
----Epoch[1],Batch[649]----
Loss after 83003 examples: 0.604
Acc after 83003 examples: 0.9788
f1_score af

...

Accuracy of the model on the 8037 test data: 95.18477043673012%
F1-Score of the model on the 8037 test data:0.0203
AUC of the model on the 8037 test data:0.5325
----Epoch[20],Batch[7324]----
Loss after 936092 examples: 0.434
Acc after 936092 examples: 0.9518
f1_score after 936092 examples: 0.0203
AUC after 936092 examples: 0.5325
<-------------------------------------->
Accuracy of the model on the 8037 test data: 91.19074281448302%
F1-Score of the model on the 8037 test data:0.0354
AUC of the model on the 8037 test data:0.5367
----Epoch[20],Batch[7349]----
Loss after 939292 examples: 0.426
Acc after 939292 examples: 0.9119
f1_score after 939292 examples: 0.0354
AUC after 939292 examples: 0.5367
<-------------------------------------->
Accuracy of the model on the 8037 test data: 93.66679109120318%
F1-Score of the model on the 8037 test data:0.0378
AUC of the model on the 8037 test data:0.5520
----Epoch[20],Batch[7374]----
Loss after 942492 examples: 0.386
Acc after 942492 examples: 0.

Accuracy of the model on the 8037 test data: 94.79905437352245%
F1-Score of the model on the 8037 test data:0.0188
AUC of the model on the 8037 test data:0.5445
----Epoch[22],Batch[7874]----
Loss after 1006354 examples: 0.445
Acc after 1006354 examples: 0.9480
f1_score after 1006354 examples: 0.0188
AUC after 1006354 examples: 0.5445
<-------------------------------------->
Accuracy of the model on the 8037 test data: 94.23914395918875%
F1-Score of the model on the 8037 test data:0.0294
AUC of the model on the 8037 test data:0.5487
----Epoch[22],Batch[7899]----
Loss after 1009554 examples: 0.428
Acc after 1009554 examples: 0.9424
f1_score after 1009554 examples: 0.0294
AUC after 1009554 examples: 0.5487
<-------------------------------------->
Accuracy of the model on the 8037 test data: 95.44606196341918%
F1-Score of the model on the 8037 test data:0.0108
AUC of the model on the 8037 test data:0.5361
----Epoch[22],Batch[7924]----
Loss after 1012754 examples: 0.427
Acc after 1012754 ex

Accuracy of the model on the 8037 test data: 91.97461739455021%
F1-Score of the model on the 8037 test data:0.0444
AUC of the model on the 8037 test data:0.5225
----Epoch[23],Batch[8424]----
Loss after 1076685 examples: 0.325
Acc after 1076685 examples: 0.9197
f1_score after 1076685 examples: 0.0444
AUC after 1076685 examples: 0.5225
<-------------------------------------->
Accuracy of the model on the 8037 test data: 92.48475799427648%
F1-Score of the model on the 8037 test data:0.0533
AUC of the model on the 8037 test data:0.5384
----Epoch[23],Batch[8449]----
Loss after 1079885 examples: 0.376
Acc after 1079885 examples: 0.9248
f1_score after 1079885 examples: 0.0533
AUC after 1079885 examples: 0.5384
<-------------------------------------->
Accuracy of the model on the 8037 test data: 94.71195719795944%
F1-Score of the model on the 8037 test data:0.0449
AUC of the model on the 8037 test data:0.5504
----Epoch[23],Batch[8474]----
Loss after 1083085 examples: 0.462
Acc after 1083085 ex

Accuracy of the model on the 8037 test data: 94.05250715441085%
F1-Score of the model on the 8037 test data:0.0324
AUC of the model on the 8037 test data:0.5519
----Epoch[25],Batch[8974]----
Loss after 1146947 examples: 0.392
Acc after 1146947 examples: 0.9405
f1_score after 1146947 examples: 0.0324
AUC after 1146947 examples: 0.5519
<-------------------------------------->
Accuracy of the model on the 8037 test data: 93.28107502799553%
F1-Score of the model on the 8037 test data:0.0391
AUC of the model on the 8037 test data:0.5586
----Epoch[25],Batch[8999]----
Loss after 1150147 examples: 0.377
Acc after 1150147 examples: 0.9328
f1_score after 1150147 examples: 0.0391
AUC after 1150147 examples: 0.5586
<-------------------------------------->
Accuracy of the model on the 8037 test data: 88.80179171332587%
F1-Score of the model on the 8037 test data:0.0426
AUC of the model on the 8037 test data:0.5324
----Epoch[25],Batch[9024]----
Loss after 1153347 examples: 0.466
Acc after 1153347 ex

Accuracy of the model on the 8037 test data: 91.81286549707602%
F1-Score of the model on the 8037 test data:0.0380
AUC of the model on the 8037 test data:0.5554
----Epoch[26],Batch[9524]----
Loss after 1217278 examples: 0.428
Acc after 1217278 examples: 0.9181
f1_score after 1217278 examples: 0.0380
AUC after 1217278 examples: 0.5554
<-------------------------------------->
Accuracy of the model on the 8037 test data: 89.48612666417817%
F1-Score of the model on the 8037 test data:0.0430
AUC of the model on the 8037 test data:0.5358
----Epoch[26],Batch[9549]----
Loss after 1220478 examples: 0.387
Acc after 1220478 examples: 0.8949
f1_score after 1220478 examples: 0.0430
AUC after 1220478 examples: 0.5358
<-------------------------------------->
Accuracy of the model on the 8037 test data: 85.70362075401269%
F1-Score of the model on the 8037 test data:0.0401
AUC of the model on the 8037 test data:0.5461
----Epoch[26],Batch[9574]----
Loss after 1223678 examples: 0.404
Acc after 1223678 ex